In [1]:
import cv2
import numpy as np
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
classes = []
with open('coco.names','r') as f:
    classes = f.read().splitlines()
# printing the data which is loaded from the names file
#print(classes)
img = cv2.imread('image.jpg')
height, width, _ = img.shape
blob = cv2.dnn.blobFromImage(img, 1/255,  (416 , 416), (0,0,0) ,1,crop=False)
net.setInput(blob)
output_layer_names = net.getUnconnectedOutLayersNames()
layerOutput = net.forward(output_layer_names)
boxes = []
confidences = []
class_ids =[]
chairs=0
bottles =0 
diningtable = 0
# cv2.imshow('Image',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [2]:
for output in layerOutput:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)


            x = int(center_x - w/2)
            y = int(center_y - h/2)

            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

In [3]:
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
font = cv2.FONT_HERSHEY_COMPLEX
colors = np.random.uniform(0,255,size=(len(boxes),3))

In [4]:
for i in indexes.flatten():
    labelsss = str(classes[class_ids[i]])
    if(labelsss == 'chair'):
        chairs+=1
    if(labelsss == 'diningtable'):
        diningtable+=1

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label =str(classes[class_ids[i]])
    confidence = str(round(confidences[i],1))
    color = colors[i]
    cv2.rectangle(img,(x,y),(x+w , y+h), color, 2)
   
    # print("label"+label)
    cv2.putText(img, label + " ", (x, y+20), font, 0.8, (0,0,0),2)
    cv2.putText(img, 'Chair'+ ":" + str(chairs), (20, 20), font, 0.8, (0,0,0),2)
    cv2.putText(img, 'Dining Table'+ ":" + str(diningtable), (20, 40), font, 0.8, (0,0,0),2)
   # cv2.putText(img, 'Chair'+ ":" + chairs, (20, 20), font, 0.8, (0,0,0),2)

In [5]:
cv2.imshow('Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()